In [1]:
# installing dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
!ls /usr/lib/jvm/
!pip install -U pyarrow
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

!ls /content/spark-3.5.0-bin-hadoop3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f46608e0d79b5ed4225e1ed653a69c2458ce67d0cc4727a945d437bc2fdd0cf8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
java-1.11.0-openjdk-amd64  java-11-openjdk-amd64  java-1.8.0-openjdk-amd64  java-8-openjdk-amd64
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 6.2.0 requires pyarrow<13,>=2, but you have pyarrow 14.0.1 

In [2]:
# cloning tpch repository
!git clone https://github.com/ssavvides/tpch-spark


Cloning into 'tpch-spark'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 557 (delta 19), reused 26 (delta 7), pack-reused 501
Receiving objects: 100% (557/557), 1.82 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [3]:
!cd /content/tpch-spark/dbgen && make


gcc -g -DDBNAME=\"dss\" -DLINUX -DORACLE -DTPCH -DRNG_TEST -D_FILE_OFFSET_BITS=64   -c -o build.o build.c
In file included from build.c:45:
build.c: In function ‘mk_time’:
dss.h:494:16: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘long int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
  494 |   sprintf(tgt, "19%02d-%02d-%02d", yr, mn, dy)
      |                ^~~~~~~~~~~~~~~~~~
build.c:422:9: note: in expansion of macro ‘PR_DATE’
  422 |         PR_DATE(t->alpha, y, m,
      |         ^~~~~~~
dss.h:494:22: note: format string is defined here
  494 |   sprintf(tgt, "19%02d-%02d-%02d", yr, mn, dy)
      |                   ~~~^
      |                      |
      |                      int
      |                   %02ld
dss.h:494:16: warning: format ‘%d’ expects argument of type ‘int’, but argument 4 has type ‘long int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wforma

In [4]:
%cd /content/tpch-spark/dbgen
!./dbgen


/content/tpch-spark/dbgen
TPC-H Population Generator (Version 2.17.0)
Copyright Transaction Processing Performance Council 1994 - 2010


In [5]:
# Specify the directory path
directory_path = "/content/tpch-spark/dbgen"

# List files with the extension .tbl in the directory
tbl_files = [file for file in os.listdir(directory_path) if file.endswith(".tbl")]

# Display the list of .tbl files
tbl_files


['nation.tbl',
 'orders.tbl',
 'supplier.tbl',
 'partsupp.tbl',
 'region.tbl',
 'lineitem.tbl',
 'part.tbl',
 'customer.tbl']

In [6]:

# Import necessary modules
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.master("local[*]").appName("TPC-H Data Loading").getOrCreate()

# Specify the directory path containing .tbl files
tbl_directory = "/content/tpch-spark/dbgen"

# List .tbl files in the directory
tbl_files = [file for file in os.listdir(tbl_directory) if file.endswith(".tbl")]

# Read each .tbl file into a PySpark DataFrame
table_dataframes = {}
for tbl_file in tbl_files:
    table_name = tbl_file.split(".")[0]
    table_path = os.path.join(tbl_directory, tbl_file)
    table_dataframes[table_name] = spark.read.option("delimiter", "|").csv(table_path, header=False, inferSchema=True)

# Save each DataFrame as a CSV file
output_csv_directory = "/content/tpch-spark/csv_output"
for table_name, df in table_dataframes.items():
    output_csv_path = os.path.join(output_csv_directory, f"{table_name}.csv")
    df.write.csv(output_csv_path, header=True, mode="overwrite")

# Display the first few rows of each DataFrame
for table_name, df in table_dataframes.items():
    print(f"\nTable: {table_name}")
    df.show(5)

# Stop the Spark session
spark.stop()



Table: nation
+---+---------+---+--------------------+----+
|_c0|      _c1|_c2|                 _c3| _c4|
+---+---------+---+--------------------+----+
|  0|  ALGERIA|  0| haggle. carefull...|NULL|
|  1|ARGENTINA|  1|al foxes promise ...|NULL|
|  2|   BRAZIL|  1|y alongside of th...|NULL|
|  3|   CANADA|  1|eas hang ironic, ...|NULL|
|  4|    EGYPT|  4|y above the caref...|NULL|
+---+---------+---+--------------------+----+
only showing top 5 rows


Table: orders
+---+------+---+---------+----------+--------+---------------+---+--------------------+----+
|_c0|   _c1|_c2|      _c3|       _c4|     _c5|            _c6|_c7|                 _c8| _c9|
+---+------+---+---------+----------+--------+---------------+---+--------------------+----+
|  1| 36901|  O|173665.47|1996-01-02|   5-LOW|Clerk#000000951|  0|nstructions sleep...|NULL|
|  2| 78002|  O| 46929.18|1996-12-01|1-URGENT|Clerk#000000880|  0| foxes. pending a...|NULL|
|  3|123314|  F|193846.25|1993-10-14|   5-LOW|Clerk#000000955|  0|

In [7]:
import shutil

In [8]:
# Zip the CSV files
shutil.make_archive("/content/tpch-spark/csv_output_archive", 'zip', "/content/tpch-spark/csv_output/customer.csv")


'/content/tpch-spark/csv_output_archive.zip'

In [9]:
# Zip the CSV files
shutil.make_archive("/content/tpch-spark/csv_output_orders", 'zip', "/content/tpch-spark/csv_output/orders.csv")


'/content/tpch-spark/csv_output_orders.zip'

In [10]:
# Zip the CSV files
shutil.make_archive("/content/tpch-spark/csv_output_archive", 'zip', output_csv_directory)


'/content/tpch-spark/csv_output_archive.zip'

In [11]:
# Zip the CSV files
shutil.make_archive("/content/tpch-spark/csv_output_nation", 'zip', "/content/tpch-spark/csv_output/nation.csv")


'/content/tpch-spark/csv_output_nation.zip'